In [1]:
##This code is for finding minimal value for training data by convex formulation based on the paper
##"Neural Networks are Convex Regularizers: Exact Polynomial-time Convex Optimization Formulations for Two-layer Networks"
##regression (squared loss), fixed design. response y is random
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import torch

In [83]:
def relu(x):
    return np.maximum(0,x)
def drelu(x):
    return x>=0
n=5 # number of samples
d=2 # dimension of the data
beta=1e-3 # regularization parameter
dv=1 #standard deviation of noise

In [84]:
X = torch.tensor([[2.0, 1.0],
                  [1.0, 2.0],
                  [2.0, 4.0],
                  [4.0, 2.0],
                  [-1, 1.0]], dtype=torch.float32)
# Number of rows in X
n = X.size(0)

# Create a column of ones
ones_column = torch.ones((n, 1))

# Concatenate the ones column at the front of X
#X = torch.cat((ones_column, X), dim=1)

Y = torch.tensor([1.0, -1.0, -1.0, 1.0, -1.0], dtype=torch.float32)
# Fix the random seed for reproducibility
torch.manual_seed(42)  # You can use any integer value here
# Generate Gaussian noise with the same shape as Y (mean=0, std=dv)
noise = dv*torch.randn_like(Y)

# Add Gaussian noise to Y
Y = Y + noise

In [85]:
print(X)
print(Y)

tensor([[ 2.,  1.],
        [ 1.,  2.],
        [ 2.,  4.],
        [ 4.,  2.],
        [-1.,  1.]])
tensor([ 1.3367, -0.8712, -0.7655,  1.2303, -2.1229])


In [86]:
# 1. Generate random dataset with n samples

#X = torch.rand(n, 2) * 5 - 5 # Random 2D input in range [-5, 5)
# Create Y based on the first element of each row in X
#Y = torch.where(X[:, 0] >X[:,1] , torch.tensor(1), torch.tensor(-1))
#X = torch.normal(mean=0, std=1,size=(n,1))
#X = torch.cat((X, torch.ones(n, 1)), 1)
#Y = torch.normal(mean=1, std=1, size=(n,1))
#Y = torch.rand(n, 1)

In [87]:
Y=Y.numpy()

In [88]:
##Check
print(X.shape)
print(Y.shape)
print(type(X))
print(type(Y))
print(X[:10])
print(Y[:10])

torch.Size([5, 2])
(5,)
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
tensor([[ 2.,  1.],
        [ 1.,  2.],
        [ 2.,  4.],
        [ 4.,  2.],
        [-1.,  1.]])
[ 1.3366904 -0.8711906 -0.7655376  1.2303331 -2.1228564]


In [89]:
dmat=np.empty((n,0))

In [90]:
print(dmat)
print(dmat.shape)

[]
(5, 0)


In [91]:
## Finite approximation of all possible sign patterns
for i in range(int(1e5)):
    u=np.random.randn(d,1)
    #u is a 2D array of shape (d, 1) filled with random floats sampled from a standard normal distribution (mean = 0, standard deviation = 1)
    dmat=np.append(dmat,drelu(np.dot(X,u)),axis=1)
    #The function appends the result of drelu(np.dot(X, u)) as a new column to the right of the dmat array.

dmat=(np.unique(dmat,axis=1))

In [92]:
#Ensure that dmat.shape is (5,6) for this example
#If not, increase the range for i
print(dmat.shape)

(5, 6)


In [93]:
# Optimal CVX
m1=dmat.shape[1] #m1=number of columns in dmat
#Uopt1 and Uopt2 to represent v and t to be optimised
Uopt1=cp.Variable((d,m1))
Uopt2=cp.Variable((d,m1))

In [94]:
#A symbolic representation of a quantity in Python refers to a mathematical expression or object that represents a value without specifying the actual value.
# Instead of directly working with numbers, symbolic representations allow you to manipulate, simplify, and solve mathematical expressions symbolically.
print(Uopt1)
print(m1)
print(Uopt1.shape)

var409
6
(2, 6)


In [95]:
yopt1=cp.Parameter((n,1))
yopt2=cp.Parameter((n,1))


In [96]:
#yopt1 and yopt2 to represent a fixed vector DXv and DXt
yopt1=cp.sum(cp.multiply(dmat,(X*Uopt1)),axis=1)
yopt2=cp.sum(cp.multiply(dmat,(X*Uopt2)),axis=1)

/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 17 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 18 times so far.

  warnings.warn(msg, UserWarning)


In [97]:
## Below we use hinge loss as a performance metric for binary classification
cost=cp.sum((Y - (yopt1 - yopt2))**2)/n + (beta/2)*(cp.norm(Uopt1, 'fro')**2 + cp.norm(Uopt2, 'fro')**2)

In [98]:
constraints=[]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt1))>=0]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt2))>=0]

/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 19 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cvxpy/expressions/expression.py:667: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 20 times so far.

  warnings.warn(msg, UserWarning)


In [99]:
prob=cp.Problem(cp.Minimize(cost),constraints)
prob.solve()
cvx_opt=prob.value

In [100]:
print("Convex program objective value: ",cvx_opt)

Convex program objective value:  0.121892128874057
